In [1]:
"""
Main script to run for training the RL models
"""
import os
import time
import random
from logger import create_logger
from rewards.reward_func import StepReward
import sympy as sp
import numpy as np
from observations.spaces import StringObs, ActionSpace
from envs.gym_env import PolyLogLinExpr
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.logger import configure
from model.rl_algorithms import TRPOModel
from model.monitor_utils import SaveOnBestTrainingRewardCallback, plot_results, plot_all_results, import_curriculum

/home/stringer/anaconda3/envs/MLenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

if __name__ == '__main__':

    logger = create_logger('run.log')
    logger.info("============ Initialized logger ============")

    # Define the training timesteps along with the episode length
    max_step = 50
    total_t_steps = 3000000
    check_freq = 30000

    # Reward function to use
    reward_func = StepReward(len_pen=0.00, num_terms_pen=0.0, active_pen=False, simple_reward=1, pen_cycle=False)

    # Use a sympy observation environment
    x = sp.Symbol('x')
    expr0 = 2 * sp.polylog(2, x)
    obs_space = StringObs(expr0, 512, one_hot_encode=True, numeral_decomp=True, reduced_graph=True, prev_st_info=True)

    # Use an Discrete action space with 3 actions
    act_space = ActionSpace(['inversion', 'reflection', 'cyclic', 'duplication'])

    # At each new episode we get a new equation
    random_start = True

    # If we want to do curriculum learning
    curriculum_dir = None

    # Define the parameters for the data generation if we desire to do it on the fly
    start_size = 13500  # Training set size
    len_simple = 0
    len_scr = 4
    num_scr = 7

    # Otherwise we can load a previously generated training set
    file_starts = '/home/stringer/下载/data_and_model/train_data.txt'

    random_start_args = {'random_start': random_start, 'start_size': start_size, 'len_simple': len_simple,
                         'len_scr': len_scr, 'num_scr': num_scr, 'file_starts': file_starts,
                         'curriculum': import_curriculum(curriculum_dir)}

    # Directory to store result
    log_dir = "log_dir"

    # RL Algorithm

    # Network architecture
    policy = 'MultiInputPolicy'
    shared_layers = [256]
    vf_layers = [128, 128, 64]
    pi_layers = [128, 128, 64]
    act_func = 'relu'

    # Agent parameters
    model_name = 'trpo'
    trpo_kwargs = {'gae_lambda': 0.9, 'gamma': 0.9, 'n_steps': 2048}
    ft_extrac = {'name': 'sage', 'params': {'embed_dim': 64, 'num_layers': 2, 'obs_space': obs_space,
                                            'bidirectional': True}}
    # ft_extrac = 'Flatten'

    num_experiments = 1
    times = []
    log_dirs = []

    # Can repeat experiment with different seeds
    for experiment in range(num_experiments):
        np.random.seed(experiment)
        random.seed(experiment)
        rng = np.random.RandomState(experiment)

        # Path to store results
        log_link = log_dir + model_name + '_exp_{}'.format(str(experiment)) + '/'
        log_dirs.append(log_link)
        os.makedirs(log_link, exist_ok=True)

        # Define the environment and check it
        env = Monitor(PolyLogLinExpr(max_step, reward_func, obs_space, act_space, rng,
                                     random_start_args=random_start_args, log_dir=log_link, gen='V2'), log_link)
        env.reset()

        # Define the agent
        base_model = TRPOModel(env, policy, shared_layers, vf_layers, pi_layers, act_func, feature_extractor=ft_extrac,
                               seed=experiment, one_hot=obs_space.one_hot_encode, **trpo_kwargs)

        # Configure the logger
        assert base_model.get_name() == model_name
        rl_model = base_model.get_model()
        logger.info(str(rl_model.policy))
        new_logger = configure(log_link, ["stdout", "csv"])
        rl_model.set_logger(new_logger)

        # Start the timer
        start = time.time()

        # Use deterministic actions for evaluation
        eval_callback = SaveOnBestTrainingRewardCallback(check_freq=check_freq, log_dir=log_link)

        # Now start the training
        rl_model.learn(total_timesteps=total_t_steps, callback=eval_callback)

        times.append(time.time() - start)

        # Save the final model and associated stats
        rl_model.save(log_link + 'final_model')
        stats_path = os.path.join(log_link, "vec_normalize_" + str(experiment) + ".pkl")

        # Plot the rewards as a function of time steps
        plot_results(log_link, window_size=int(check_freq/20))

    plot_all_results(log_dirs, log_dir + model_name + '_rewards.pdf', window_size=int(check_freq/20))


INFO - 01/25/24 20:43:09 - 0:00:00 - ============ Initialized logger ============
INFO - 01/25/24 20:43:09 - 0:00:00 - We create a reward/penalty at each step
INFO - 01/25/24 20:43:09 - 0:00:00 - Reward function has a penalty each step. Number of terms times -0.0. We also penalize lengthy expressions with -0.0 per word/arg
INFO - 01/25/24 20:43:09 - 0:00:00 - Reward function also rewards each simplification with +1
INFO - 01/25/24 20:43:09 - 0:00:00 - Final expression not simplified are penalized with 0
INFO - 01/25/24 20:43:09 - 0:00:00 - Create an Observation space of size 512
INFO - 01/25/24 20:43:09 - 0:00:00 - We use a numeral decomposition for the integers
INFO - 01/25/24 20:43:09 - 0:00:00 - We use ['x'] variables
INFO - 01/25/24 20:43:09 - 0:00:00 - We use dict_keys(['add', 'mul', 'div', 'pow', 'polylog']) operators
INFO - 01/25/24 20:43:09 - 0:00:00 - We use ['INT+', 'INT-'] symbols
INFO - 01/25/24 20:43:09 - 0:00:00 - We use ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 

Using cuda device
Wrapping the env in a DummyVecEnv.


INFO - 01/25/24 20:55:22 - 0:12:12 - MultiInputActorCriticPolicy(
                                       (features_extractor): CombinedFeatureExtractor(
                                         (word_encoder): GSageEncoder(
                                           (embeddings): Identity()
                                           (gcn_forward): GraphSAGE(20, 64, num_layers=2)
                                           (gcn_fwd_state): GraphSAGE(20, 64, num_layers=2)
                                         )
                                       )
                                       (mlp_extractor): MlpExtractor(
                                         (shared_net): Sequential(
                                           (0): Linear(in_features=131, out_features=256, bias=True)
                                           (1): ReLU()
                                         )
                                         (policy_net): Sequential(
                                       

Logging to log_dirtrpo_exp_0/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 45.8     |
|    ep_rew_mean     | 0.75     |
| time/              |          |
|    fps             | 23       |
|    iterations      | 1        |
|    time_elapsed    | 88       |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 43.2     |
|    ep_rew_mean            | 0.798    |
| time/                     |          |
|    fps                    | 17       |
|    iterations             | 2        |
|    time_elapsed           | 237      |
|    total_timesteps        | 4096     |
| train/                    |          |
|    explained_variance     | -2.21    |
|    is_line_search_success | 1        |
|    kl_divergence_loss     | 0.00603  |
|    learning_rate          | 0.001    |
|    n_updates              | 1        |
|    policy_obj

INFO - 01/25/24 21:33:00 - 0:49:50 - Num timesteps: 30000
INFO - 01/25/24 21:33:00 - 0:49:50 - Best mean reward: -inf - Last mean reward per episode: 0.93
INFO - 01/25/24 21:33:00 - 0:49:50 - Saving new best model to log_dirtrpo_exp_0/best_model.zip


...

INFO - 01/26/24 15:39:40 - 18:56:31 - Num timesteps: 1710000
INFO - 01/26/24 15:39:40 - 18:56:31 - Best mean reward: 2.00 - Last mean reward per episode: 1.72


----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 26.6     |
|    ep_rew_mean            | 1.75     |
| time/                     |          |
|    fps                    | 25       |
|    iterations             | 835      |
|    time_elapsed           | 67460    |
|    total_timesteps        | 1710080  |
| train/                    |          |
|    explained_variance     | 0.433    |
|    is_line_search_success | 1        |
|    kl_divergence_loss     | 0.00956  |
|    learning_rate          | 0.001    |
|    n_updates              | 834      |
|    policy_objective       | 0.0161   |
|    value_loss             | 0.145    |
----------------------------------------
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 28       |
|    ep_rew_mean            | 1.8      |
| time/                     |          |
|    fps                    | 25       |
|    iterations 

INFO - 01/26/24 16:00:46 - 19:17:36 - Num timesteps: 1740000
INFO - 01/26/24 16:00:46 - 19:17:36 - Best mean reward: 2.00 - Last mean reward per episode: 1.66


----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 23.1     |
|    ep_rew_mean            | 1.85     |
| time/                     |          |
|    fps                    | 25       |
|    iterations             | 850      |
|    time_elapsed           | 68739    |
|    total_timesteps        | 1740800  |
| train/                    |          |
|    explained_variance     | 0.34     |
|    is_line_search_success | 1        |
|    kl_divergence_loss     | 0.00827  |
|    learning_rate          | 0.001    |
|    n_updates              | 849      |
|    policy_objective       | 0.0129   |
|    value_loss             | 0.149    |
----------------------------------------
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 27.1     |
|    ep_rew_mean            | 1.98     |
| time/                     |          |
|    fps                    | 25       |
|    iterations 

KeyboardInterrupt: 